# UCB PoC 
- WaferFixMatch 후속 연구

- 가정 
   - 웨이퍼빈맵 불량 패턴별 적합한 augmentation들은 각각 따로 있을 것
   - WaferFixMatch에 해당 하는 Weak와 Strong에 각각에 augmentation이 적합하게 부여되도록 필요
   - WaferFixMatch Saliency map용 인코더 이용하여 레이블을 얻어 해당 패턴 UCB로 매핑하는 방법
   - 5% 레이블 활용하고 그 데이터를 지도학습하고 얻은 레이블을 이용하여 UCB에 할당하고 데이터를 증강시도 하고 repreentation vector에 대한 유사도를 
   - UCB에서 Q(a)에 대한 정의를 reprsentation vector 유사도 및 모델 정확도도 같이 보기 위해서 레이블을 맞추었냐가 들어가야하는데 semi에 어디 부분에 kick in이 될 것인가?
   - rotation invariant이 보장되므로 손실 함수에 term을 추가하여 학습 (강화학습 아님) -> 샘플 증강하고 불량패턴 같음을 보장

- 적용 아이디어
   - 5% 레이블 활용하여 사전 학습된 모델 예측 레이블을 사전에 저장하고 
   - 미니 배치 단위 학습 추정한 레이블을 넣는데 
   - 배치마다 층화 샘플링을 하더라도 미니배치 내 각 패턴별 분포를 알기 어려움
   - 미니 배치에 예를 들어 3개의 클래스가 있는 데이터가 1번이 10개 2번이 20개 3번이 15개가 있고 각 유사도를 얻고 
   - 증강된 데이터를 이용하여 실제 학습되고 있는 모델이 같은 레이블로 예측에 성공을 할 경우 추가 weight를 주는것을 반영할 수 있을듯 하다.
   - UCB 알고리즘에 Contextual 적용여부
     - 사용 : Contextual armed bandit 사용, context = 불량 패턴 레이블 예측값 (아님 입력 데이터 -> Heavy?) 
     - 미사용 : 각 패턴마다 별도의 UCB bandit을 두도록 셋팅하는 것도 방안 -> pretrained된 모델이 잘되어야?
   --> pretrained model에서 추측한 레이블 정보에 의존하는 문제를 어떻게 돌파할 것인가,,
   - https://github.com/etiennekintzler/visualize_bandit_algorithms/tree/master
   - context를 32x32 입력 사이즈 그대로 해보자.  
   - The use of contextual multi-armed bandits (MABs) with a context size of 1024 is certainly feasible!!
   
   - batch 단위로 Q(a)가 아래와 같이 계산되고 update가 되도록 
     - weak augmentation bandit   : original 이미지와 weak augmentation된 이미지의 pretrained된 모델 출력 벡터의 cosine similirity 를 계산 + weak label과 strong label이 일치할 경우 추가 리워드 부여 
     - strong augmentation bandit : weak와 strong의 cosine similiarity와 유사해야 함 + weak label과 strong label이 일치할 경우 추가 리워드 부여 

   - pretrained model이 없어도 가능할듯
      - 불량 패턴마다 적합한 augmentation이 다를 것이다에서 contextual을 class가 필요한데
      - 불량 분류 패턴에서 이미지 자체를 context로 바라보면 simple
      - hybrid bandit 이 아닌 disjoint bandit을 사용하여 구현

- 구현체
   - 기존 WaferFixMatch 모델 Backbone 그대로 사용
   - 데이터로더는 UCB에서 주는 action (augmentation 기법)에 따라 데이터 로드되도록
   - linear UCB를 사용해서 contextual한 정보 활용 가능하게 하거나
   - epslion-greeedy가 되었던 아니면 UCB 를 각 패턴별로 오브젝트를 만들어서 독립적인 학습 진행 (적은 양의 패턴 경우, 감가율도 보정)
   - 학습 데이터를 이용하여 아예 레이블 정보를 가지고 supervised contrastive learning 처럼 아예 답안을 가지고 하듯이 선행 UCB학습한 것을 이용하는것도 방법( two stage )


# LinUCB PoC

In [1]:
# https://yjjo.tistory.com/44


import requests
import numpy as np

url = "http://www.cs.columbia.edu/~jebara/6998/dataset.txt"

# Make the GET request to download the file content
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request was unsuccessful

# Save the content to a local file named 'data.txt'
with open("data.txt", "w", encoding='utf-8') as file:
    file.write(response.text)

print("Content saved to data.txt!")


Content saved to data.txt!


In [8]:
# Create class object for a single linear ucb disjoint arm
class linucb_disjoint_arm():
    
    def __init__(self, arm_index, d, alpha):

        # Track arm index
        self.arm_index = arm_index
        
        # Keep track of alpha
        self.alpha = alpha
        
        # A: (d x d) matrix = D_a.T * D_a + I_d. 
        # The inverse of A is used in ridge regression 
        self.A = np.identity(d)
        
        # b: (d x 1) corresponding response vector. 
        # Equals to D_a.T * c_a in ridge regression formulation
        self.b = np.zeros([d,1])
        
    def calc_UCB(self, x_array):
        # Find A inverse for ridge regression
        A_inv = np.linalg.inv(self.A)
        
        # Perform ridge regression to obtain estimate of covariate coefficients theta
        # theta is (d x 1) dimension vector
        self.theta = np.dot(A_inv, self.b)
        
        # Reshape covariates input into (d x 1) shape vector
        x = x_array.reshape([-1,1])
        
        # Find ucb based on p formulation (mean + std_dev) 
        # p is (1 x 1) dimension vector
        p = np.dot(self.theta.T,x) +  self.alpha * np.sqrt(np.dot(x.T, np.dot(A_inv,x)))
        
        return p
    
    def reward_update(self, reward, x_array):
        # Reshape covariates input into (d x 1) shape vector
        x = x_array.reshape([-1,1])
        
        # Update A which is (d * d) matrix.
        self.A += np.dot(x, x.T)
        
        # Update b which is (d x 1) vector
        # reward is scalar
        self.b += reward * x

In [9]:
class linucb_policy():
    
    def __init__(self, K_arms, d, alpha):
        self.K_arms = K_arms
        self.linucb_arms = [linucb_disjoint_arm(arm_index = 1, d = d, alpha = alpha) for i in range(K_arms)]
        
    def select_arm(self, x_array):
        # Initiate ucb to be 0
        highest_ucb = -1
        
        # Track index of arms to be selected on if they have the max UCB.
        candidate_arms = []
        
        for arm_index in range(self.K_arms):
            # Calculate ucb based on each arm using current covariates at time t
            arm_ucb = self.linucb_arms[arm_index].calc_UCB(x_array)
            
            # If current arm is highest than current highest_ucb
            if arm_ucb > highest_ucb:
                
                # Set new max ucb
                highest_ucb = arm_ucb
                
                # Reset candidate_arms list with new entry based on current arm
                candidate_arms = [arm_index]

            # If there is a tie, append to candidate_arms
            if arm_ucb == highest_ucb:
                
                candidate_arms.append(arm_index)
        
        # Choose based on candidate_arms randomly (tie breaker)
        chosen_arm = np.random.choice(candidate_arms)
        
        return chosen_arm

In [10]:
def ctr_simulator(K_arms, d, alpha, data_path):
    # Initiate policy
    linucb_policy_object = linucb_policy(K_arms = K_arms, d = d, alpha = alpha)
    
    # Instantiate trackers
    aligned_time_steps = 0
    cumulative_rewards = 0
    aligned_ctr = []
    unaligned_ctr = [] # for unaligned time steps
    
    # Open data
    with open(data_path, "r") as f:

        for line_data in f:

            # 1st column: Logged data arm. 
            # Integer data type
            data_arm = int(line_data.split()[0])

            # 2nd column: Logged data reward for logged chosen arm
            # Float data type
            data_reward = float(line_data.split()[1])

            # 3rd columns onwards: 100 covariates. Keep in array of dimensions (100,) with float data type
            covariate_string_list = line_data.split()[2:]
            data_x_array = np.array([float(covariate_elem) for covariate_elem in covariate_string_list])

            # Find policy's chosen arm based on input covariates at current time step
            arm_index = linucb_policy_object.select_arm(data_x_array)

            # Check if arm_index is the same as data_arm (ie same actions were chosen)
            # Note that data_arms index range from 1 to 10 while policy arms index range from 0 to 9.
            if arm_index + 1 == data_arm:

                # Use reward information for the chosen arm to update
                linucb_policy_object.linucb_arms[arm_index].reward_update(data_reward, data_x_array)

                # For CTR calculation
                aligned_time_steps += 1
                cumulative_rewards += data_reward
                aligned_ctr.append(cumulative_rewards/aligned_time_steps)
                    
    return (aligned_time_steps, cumulative_rewards, aligned_ctr, linucb_policy_object)

In [11]:
alpha_input = 1.5
data_path = "data.txt"
aligned_time_steps, cum_rewards, aligned_ctr, policy = ctr_simulator(K_arms = 10, d = 100, alpha = alpha_input, data_path = data_path)

IndexError: list index out of range

In [3]:
import os
import logging
import time
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
import wandb
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from datasets.samplers import ImbalancedDatasetSampler
from tqdm import tqdm
from datasets.dataset import DATASET_GETTERS
from datasets.dataset import WM811K
from utils import AverageMeter, accuracy
from utils.common import get_args, de_interleave, interleave, save_checkpoint
from utils.common import set_seed, create_model, get_cosine_schedule_with_warmup
from datetime import datetime
import yaml
from argparse import Namespace
from PIL import Image
import collections
import pandas as pd
# from tabulate import tabulate
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
torch.cuda.set_device(0)

In [ ]:
labeled_dataset, unlabeled_dataset, valid_dataset, test_dataset = DATASET_GETTERS[args.dataset](args, './data')